In [3]:
# importa modulos
%pylab
import pickle
import numpy as np
import itertools
import matplotlib.pyplot as plt
from scipy import signal
import pandas as pd

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [5]:
#Cambios Sergio

#RAT01
#ruta ='/home/sergio/Documentos/MemoriaPractII/DatosCaro/RR01/EC01_RRR01_SP_12600uA__2018-01-03_13-09-47/'
#fname='LFP_EC01_RRR01_SP_12600uA__20180103'

#RAT02
#ruta ='/home/sergio/Documentos/MemoriaPractII/DatosCaro/RAT02/RAT02_SP_7__2017-01-11_18-46-49/'
#fname='LFP_RAT02_SP_7__20170111_184649'

#RAT03
#ruta ='/home/sergio/Documentos/MemoriaPractII/DatosCaro/RRR03/EC01_RRR03_SP_3500uA_2017-12-15_11-45-36/'
#fname='LFP_EC01_RRR03_SP_3500uA_20171215'

#RAT05
#ruta ='/home/sergio/Documentos/MemoriaPractII/DatosCaro/RC05/EC01_RC05_SP_10100uA_2017-12-14_17-00-09/'
#fname='LFP_EC01_RC05_SP_10100uA_20171214'

## En computador registro 2
#RAT01
ruta = '/home/controlmotor/SERGIO/DatosCaro/RR01/EC01_RRR01_SP_12600uA__2018-01-03_13-09-47/'
fname='LFP_EC01_RRR01_SP_12600uA__20180103'

#RAT02
#ruta ='/home/controlmotor/SERGIO/DatosCaro/RAT02/RAT02_SP_7__2017-01-11_18-46-49/'
#fname='LFP_RAT02_SP_7__20170111_184649'

#RAT03
#ruta ='/home/controlmotor/SERGIO/DatosCaro/RRR03/EC01_RRR03_SP_3500uA_2017-12-15_11-45-36/'
#fname='LFP_EC01_RRR03_SP_3500uA_20171215'

#RAT05
#ruta ='/home/controlmotor/SERGIO/DatosCaro/RC05/EC01_RC05_SP_10100uA_2017-12-14_17-00-09/'
#fname='LFP_EC01_RC05_SP_10100uA_20171214'
data = pickle.load(open(ruta+fname+'.pickle', "rb" ) )
lfp = data[0]


### Versión fácil para extraer los canales de estimulación.

In [ ]:
# Extraer canal con los pulsos de estimulacion, siempre será el canal ADC8 
import sys
sys.path.append('/home/sergio/Documentos/MemoriaPractII/Open Ephys/analysis-tools-master')
import OpenEphys as OE

ttl_channel = '100_ADC8' 
extension = '.continuous'
ttl_channel = ttl_channel+extension
data = OE.loadContinuous(ruta+ttl_channel)
ttl_signal = data['data']
sampleRate = double(data['header']['sampleRate'])
# construye un vector de tiempo con el samplerate
t0 = 1/sampleRate
tf = len(ttl_signal)/sampleRate + 1/sampleRate
step = 1/sampleRate
#Vector de tiempo es ttl_t (ttl_time).
ttl_t = np.arange(t0,tf,step)
# normaliza la señal de ttl, minimo 0, maximo 1
delta = max(ttl_signal)- min(ttl_signal)
ttl_signal = (ttl_signal - min(ttl_signal))/delta
# busca los indices de las muestras > o iguales a 0.8 (tolerancia de 0.2)
thr = 0.7
ind = ttl_signal>=thr
# encuentra los tiempos
time_stamps = ttl_t[ind]

flag = 0
on_ts = np.empty(0)
off_ts = np.empty(0)
for i in range(0,len(ttl_signal)):
    if flag == 0:
        if ttl_signal[i]>thr:
            on_ts = np.append(on_ts,ttl_t[i])
            flag = 1
    if flag == 1:
        if ttl_signal[i]<thr:
            off_ts = np.append(off_ts,ttl_t[i])
            flag = 0  
len(on_ts)

In [ ]:
# save stimulation timestamps
pickle.dump([on_ts],open(ruta+'STIM_' + fname[4:] +'.pickle', "wb" ) )

In [6]:
# load stimulation timestamps
#stimfile = 'STIM_RAT02_SP_7_2017_01_11_18_46_49'
#stimfile='STIM_EC01_RRR03_SP_3500uA_20171215'
#data = pickle.load(open(ruta+stimfile+'.pickle', "rb" ) )
data = pickle.load(open(ruta+'STIM_'+fname[4:]+'.pickle', "rb" ) )
on_ts = data[0]
del data

In [4]:
on_ts

array([  40.7221    ,   42.7214    ,   44.72066667,   46.71996667,
         48.71926667,   50.71856667,   52.71786667,   54.71713333,
         56.71643333,   58.71573333,   60.71503333,   62.71433333,
         64.71363333,   66.7129    ,   68.7122    ,   70.7115    ,
         72.7108    ,   74.7101    ,   76.70936667,   78.70866667,
         80.70796667,   82.70726667,   84.70656667,   86.70583333,
         88.70513333,   90.70443333,   92.70373333,   94.70303333,
         96.7023    ,   98.7016    ,  100.7009    ,  102.7002    ,
        104.6995    ,  106.6988    ,  108.69806667,  110.69736667,
        112.69666667,  114.69596667,  116.69526667,  118.69453333,
        120.69383333,  122.69313333,  124.69243333,  126.69173333,
        128.691     ,  130.6903    ,  132.6896    ,  134.6889    ,
        136.6882    ,  138.6875    ,  140.68676667,  142.68606667,
        144.68536667,  146.68466667,  148.68396667,  150.68323333,
        152.68253333,  154.68183333,  156.68113333,  158.68043

In [5]:
%pylab
fig, ax=subplots()
ax.plot(lfp.LFP[0])

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


# Generamos el data frame con la LFP separada por ventanas, sin filtrar. 

In [10]:
rat='RAT01'
### get the LFP and save them to dataframe
Ldopa = 'off'
#N = 6
sr = lfp.SampleRate[0]
#nyq = sr/2.0
#f1 = 10.0
#f2 = 35.0
# extract sampling rate and create a time axis t
sr = lfp.SampleRate[0] # sampling rate
#print sr
#t = arange(1/sr,len(lfp.LFP[0])/sr + 1/sr,1/sr)
#len(t),len(lfp.LFP[0])
past = 0.3
future = 0.7
tt = arange(-past,future,1/sr)
# band pass freqs expressed as Nyquist
#Wn = np.array([f1/nyq, f2/nyq])
# design parameters for the butter filter
#b,a = scipy.signal.butter(N, Wn, btype='bandpass', analog=False)
#b,a = signal.butter(N, Wn, btype='bandpass', analog=False)
df = pd.DataFrame([], columns=['Rat',
                               'File',
                               'L-dopa',
                               'SampleRate',
                               #'Bandpass',
                               'tWindow',
                               'Channel',
                               'Area',
                               'UnfilteredTrialsxSamples'])
#Por canal en canales seleccionados previamente. 
for i in range(0,len(lfp)):
    #Filtrar pasa banda
    #ad = signal.filtfilt(b, a, lfp.LFP.loc[i], padlen=None)
    #LFP del canal i
    ad = lfp.LFP.loc[i]
    #Formar una matriz de TrialsxSamples
    lfparray = np.zeros([len(on_ts),int(past*sr+future*sr)])
    #Formar un arreglo de largo de los Trials para dejar aqui los promedios de las derivadas
    out = np.zeros(len(on_ts))

    # collect the trials around stimulation pulse
    #Por timestamps en el total de timestamps
    for j in range(0,len(on_ts)):
        #Obtengo el timestamp
        t1 = on_ts[j]
        #Redondeo el timestamp
        ind = int(sr*round(t1,3)-1)
        #Formo la ventana alrededor del timestamp. 0.3 al pasado y 0.7 al futuro
        adaux = ad[int(ind-past*sr):int(ind+future*sr)]
        #Lleno la fila del trial correspondiente en la matriz de TrialsxSamples
        lfparray[j,:] = adaux
        #Calculo el promedio de la derivada de dicho trial
        out[j]=np.mean(np.abs(diff(adaux)))
    #Ordeno los trials por promedio    
    sorted_out = sort(out)
    #Calculamos el limite (95%). La idea es que el 5% mas alto posee artefactos 
    lim =  sorted_out[int(len(sorted_out)-len(sorted_out)*5/100)]
    #Se eliminan los trials que pertenezcan al 5 por ciento mayor
    lfparray=lfparray[out<lim,:] 
    #Ahora la matriz es de Trials(95%)xSamples(1000)
    
    #Procedemos a guardar 
    # create small df with the data of that channel
    df2 = pd.DataFrame([[rat,
                             fname,
                             Ldopa,
                             lfp.SampleRate.loc[i],
                             #[f1,f2],
                             [-past,future],
                             lfp.Channel.loc[i],
                             lfp.Area.loc[i],
                             #lfparray.mean(0)]],
                             lfparray]],
                       columns=['Rat',
                               'File',
                               'L-dopa',
                               'SampleRate',
                               #'Bandpass',
                               'tWindow',
                               'Channel',
                               'Area',
                               'UnfilteredTrialsxSamples'])
    df = pd.concat([df, df2], axis = 0)
df=df.reset_index()
del df['index']

In [11]:
# save to pickle
new = 'Unfiltered_LFP_Trials_95_Percent_'
df.to_pickle(ruta+new+fname[4:]+'.pickle')
print ('Saving file to '+ruta+new+fname[4:]+'.pickle')

Saving file to /home/sergio/Documentos/MemoriaPractII/DatosCaro/RR01/EC01_RRR01_SP_12600uA__2018-01-03_13-09-47/Unfiltered_LFP_Trials_95_Percent_EC01_RRR01_SP_12600uA__20180103.pickle


In [7]:
new = 'Unfiltered_LFP_Trials_95_Percent_'
UnfData = pickle.load(open(ruta+new+fname[4:]+'.pickle', "rb" ) )

In [7]:
UnfData

,Rat,File,L-dopa,SampleRate,tWindow,Channel,Area,UnfilteredTrialsxSamples
0,RAT01,LFP_EC01_RRR01_SP_12600uA__20180103,off,1000.0,"[-0.3, 0.7]",CH1,rVL,"[[-0.11445760016595974, 0.00014259672846739235..."
1,RAT01,LFP_EC01_RRR01_SP_12600uA__20180103,off,1000.0,"[-0.3, 0.7]",CH10,rM1,"[[-0.33402348074531474, -0.18882449753854647, ..."
2,RAT01,LFP_EC01_RRR01_SP_12600uA__20180103,off,1000.0,"[-0.3, 0.7]",CH11,rM1,"[[-0.00825725361236418, -0.05555902442275229, ..."
3,RAT01,LFP_EC01_RRR01_SP_12600uA__20180103,off,1000.0,"[-0.3, 0.7]",CH12,rSTR,"[[-0.0385220254968045, -0.013640981714347244, ..."
4,RAT01,LFP_EC01_RRR01_SP_12600uA__20180103,off,1000.0,"[-0.3, 0.7]",CH13,rSTR,"[[-0.0519800396012261, -0.1564420807908531, -0..."
5,RAT01,LFP_EC01_RRR01_SP_12600uA__20180103,off,1000.0,"[-0.3, 0.7]",CH14,rSTR,"[[0.0057990640065739285, -0.02075311252695533,..."
6,RAT01,LFP_EC01_RRR01_SP_12600uA__20180103,off,1000.0,"[-0.3, 0.7]",CH15,rSTR,"[[0.025692211593151937, 0.007055061301846521, ..."
7,RAT01,LFP_EC01_RRR01_SP_12600uA__20180103,off,1000.0,"[-0.3, 0.7]",CH16,rVL,"[[0.08833741684767982, 0.04301496079623934, -0..."
8,RAT01,LFP_EC01_RRR01_SP_12600uA__20180103,off,1000.0,"[-0.3, 0.7]",CH17,rVPL,"[[0.15868672061046907, 0.1909046737908875, 0.1..."
9,RAT01,LFP_EC01_RRR01_SP_12600uA__20180103,off,1000.0,"[-0.3, 0.7]",CH18,rVPL,"[[-0.04064463415929853, -0.03839560623958238, ..."


In [13]:
%pylab
fig, ax=subplots()
ax.plot(lfp.LFP[0])

1000

In [8]:
import mtspec as sp
Trial = UnfData.UnfilteredTrialsxSamples[2][334,:]
sampleRate=UnfData.SampleRate[1]
A = Trial[:300]
B = Trial[300:600]
C= Trial[600:]
delta=1/sampleRate
time_bandwidth=2.0
maxf = 75
power= sp.mtspec(Trial,delta,time_bandwidth,number_of_tapers=2)
f=power[1]
power=power[0]
power2 = power[f<maxf]
F = f[f<maxf]
#plt.plot(f<maxf,power2)
FF = F[F>5]
plt.plot(F,power2)
#max_x = FF[power2.argmax()] 
#Guardar amplitud y frecuencia de 5 para arriba, scatterplot. 


In [8]:
fig, ax = plt.subplots()
plt.imshow(UnfData.UnfilteredTrialsxSamples[1])
plt.vlines(0,[0],50)
plt.colorbar()
#T = t.reshape((1, -1))
#xlabels = [s for s in t]
#ax.set_xticklabels(xlabels)
xlabelsCountour = [-0.5,-0.3, -0.1, 0.1, 0.3, 0.5, 0.7]
plt.axvline(x=300,color='r')
ax.set_xticklabels(xlabelsCountour)
plt.gca().invert_yaxis()
plt.show()

In [47]:
freq, tim, Sxx = signal.spectrogram(UnfData.UnfilteredTrialsxSamples[2][334,:], sampleRate)
plt.pcolormesh(tim, freq, Sxx)
plt.ylim([0,50])
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.colorbar()
plt.show()

In [9]:
from matplotlib.backends.backend_pdf import PdfPages
rat = 'RAT01'
#rutarat= '/home/sergio/Documentos/MemoriaPractII/Imagenes/rat01/'
rutarat = '/home/controlmotor/SERGIO/DatosCaro/RR01/'
#matA = zeros([2,950])
#matB = zeros([2,950])
#matC = zeros([2,950])
delta=1/sampleRate
time_bandwidth=2.0
maxf = 75
minf = 4
for chan in range(0,len(UnfData.Channel)):
    Channel = UnfData.UnfilteredTrialsxSamples[chan]
    #Creamos tres matrices, una por parte de ventana. Cada una guarda la tupla [valorFrecuencia, Amplitud] por parte 
    #del trial.
    matA = zeros([2,950])
    matB = zeros([2,950])
    matC = zeros([2,950])
    for i in range(0,950):
        Trial = Channel[i,:]
        A = Trial[:300]
        B = Trial[300:600]
        C= Trial[600:]
        powerA= sp.mtspec(A,delta,time_bandwidth,number_of_tapers=2)
        powerB= sp.mtspec(B,delta,time_bandwidth,number_of_tapers=2)
        powerC= sp.mtspec(C,delta,time_bandwidth,number_of_tapers=2)
        fA=powerA[1]
        fB=powerB[1]
        fC=powerC[1]
        powerA=powerA[0]
        powerB=powerB[0]
        powerC=powerC[0]
        power2A = powerA[fA<maxf]
        power2B = powerB[fB<maxf]
        power2C = powerC[fC<maxf]
        FA = fA[fA<maxf]
        FFA = FA[FA>=5]
        FB = fB[fB<maxf]
        FFB = FB[FB>=5]
        FC = fC[fC<maxf]
        FFC = FC[FC>=5]
        power3A = power2A[FA>minf]
        power3B = power2B[FB>minf]
        power3C = power2C[FC>minf]
        matA[0,i]=FFA[power3A.argmax()] 
        matA[1,i]= np.max(power3A)
        matB[0,i]= FFB[power3B.argmax()] 
        matB[1,i] = np.max(power3B)
        matC[0,i]= FFC[power3C.argmax()] 
        matC[1,i] = np.max(power3C)
    fig = plt.figure()
    line1 = plt.scatter(matA[0],matA[1], c='b', marker='x')
    line2 = plt.scatter(matB[0],matB[1], c='r', marker='^')
    line3 = plt.scatter(matC[0],matC[1], c='g', marker='.')
    plt.legend((line1, line2, line3), ('Antes', 'Durante', 'Después'), loc=2)
    plt.title('ScatterPlot Channel {} corresponding to Area {} '.format(UnfData.Channel[chan], UnfData.Area[chan]))
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('Power Spectrum [V**2/Hz]')
    plt.xlim(5, max(np.max(matA[0]),np.max(matB[0]),np.max(matC[0])))
    #plt.show()
    figname = 'ScatterPlot_MaxValueFreq_'+rat+'_'+ UnfData.Channel[chan] + '-' + UnfData.Area[chan]+ '.png'
    #print (figname) 
    #plt.show(block=False)
    #pp = PdfPages('foo.pdf')
    fig.savefig(rutarat+figname)
    print('Saving figure {}'.format(figname))
    plt.close(fig)


Saving figure ScatterPlot_MaxValueFreq_RAT01_CH1-rVL.png
Saving figure ScatterPlot_MaxValueFreq_RAT01_CH10-rM1.png
Saving figure ScatterPlot_MaxValueFreq_RAT01_CH11-rM1.png
Saving figure ScatterPlot_MaxValueFreq_RAT01_CH12-rSTR.png
Saving figure ScatterPlot_MaxValueFreq_RAT01_CH13-rSTR.png
Saving figure ScatterPlot_MaxValueFreq_RAT01_CH14-rSTR.png
Saving figure ScatterPlot_MaxValueFreq_RAT01_CH15-rSTR.png
Saving figure ScatterPlot_MaxValueFreq_RAT01_CH16-rVL.png
Saving figure ScatterPlot_MaxValueFreq_RAT01_CH17-rVPL.png
Saving figure ScatterPlot_MaxValueFreq_RAT01_CH18-rVPL.png
Saving figure ScatterPlot_MaxValueFreq_RAT01_CH2-rVL.png
Saving figure ScatterPlot_MaxValueFreq_RAT01_CH20-rS1.png
Saving figure ScatterPlot_MaxValueFreq_RAT01_CH21-rSTN.png
Saving figure ScatterPlot_MaxValueFreq_RAT01_CH22-rSTN.png
Saving figure ScatterPlot_MaxValueFreq_RAT01_CH23-rPf.png
Saving figure ScatterPlot_MaxValueFreq_RAT01_CH24-rPf.png
Saving figure ScatterPlot_MaxValueFreq_RAT01_CH25-rPf.png
Saving f

In [111]:
line1 = plt.scatter(matA[0],matA[1], c='b', marker='x')
line2 = plt.scatter(matB[0],matB[1], c='r', marker='^')
line3 = plt.scatter(matC[0],matC[1], c='g', marker='.')
#plt.scatter(matC[0],matC[1], c='k', marker='r', label='Desde hasta 0.3 despues del pulso')
plt.legend((line1, line2, line3), ('Antes', 'Durante', 'Después'), loc=2)
plt.title('Channel {} corresponding to Area {} '.format(UnfData.Channel[0], UnfData.Area[0]))
plt.xlabel('Frequency [Hz]')
plt.ylabel('Power Spectrum [V**2/Hz]')
plt.xlim(5, max(np.max(matA[0]),np.max(matB[0]),np.max(matC[0])))
plt.show()

In [12]:
from os import listdir
from fpdf import FPDF
#Computador Sergio
#path = "/home/sergio/Documentos/MemoriaPractII/Imagenes/rat01/" # get the path of images
#Computador de registros 2
path = '/home/controlmotor/SERGIO/DatosCaro/RR01/IMG/'
imagelist = listdir(path) # get list of all images
imagelist = sort(imagelist)

pdf = FPDF('P','mm','A4') # create an A4-size pdf document 

x,y,w,h = 0,0,200,150

for image in imagelist:

    pdf.add_page()
    pdf.image(path+image,x,y,w,h)
    #pdf.image(path+image,x,y+h,w,h)

pdf.output("ScatterPlot_MaxValueFreq_RAT01_.pdf","F")

''